In [1]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.applications import imagenet_utils
from keras.preprocessing.image import *

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from glob import glob
from tqdm import tqdm
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.metrics import *

import multiprocessing
from multiprocessing.dummy import Pool
from pprint import pprint

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

Using TensorFlow backend.


In [2]:
epoch = 87
batch_size = 2
model_name = 'xception'
preprocess_func = xception.preprocess_input
MODEL = Xception

f = 2
width = int(2560 / f)
height = int(1920 / f)
classes = ['正常', '吊经', '擦洞', '跳花', '毛洞', '织稀', '扎洞', '缺经', '毛斑', '其他']
n_classes = len(classes)

In [3]:
idg = ImageDataGenerator(horizontal_flip=True,
                         rotation_range=15,
                         zoom_range=0.2)
idg2 = ImageDataGenerator()

gen_train = idg.flow_from_directory('train', target_size=(height, width), 
                                    classes=classes, batch_size=batch_size)
gen_valid = idg2.flow_from_directory('valid', target_size=(height, width), shuffle=False, 
                                    classes=classes, batch_size=batch_size)
y_valid = (gen_valid.classes != 0).astype(int)

Found 1819 images belonging to 10 classes.
Found 203 images belonging to 10 classes.


In [4]:
!ls models/xception*

models/xception-10.h5  models/xception-37.h5  models/xception-63.h5
models/xception-11.h5  models/xception-38.h5  models/xception-64.h5
models/xception-12.h5  models/xception-39.h5  models/xception-65.h5
models/xception-13.h5  models/xception-3.h5   models/xception-66.h5
models/xception-14.h5  models/xception-40.h5  models/xception-67.h5
models/xception-15.h5  models/xception-41.h5  models/xception-68.h5
models/xception-16.h5  models/xception-42.h5  models/xception-69.h5
models/xception-17.h5  models/xception-43.h5  models/xception-6.h5
models/xception-18.h5  models/xception-44.h5  models/xception-70.h5
models/xception-19.h5  models/xception-45.h5  models/xception-71.h5
models/xception-1.h5   models/xception-46.h5  models/xception-72.h5
models/xception-20.h5  models/xception-47.h5  models/xception-73.h5
models/xception-21.h5  models/xception-48.h5  models/xception-74.h5
models/xception-22.h5  models/xception-49.h5  models/xception-75.h5
models/xception-23.h5  models/xception-4.h5   mod

In [5]:
model = load_model('models/%s-%d.h5' % (model_name, epoch), custom_objects={'imagenet_utils': imagenet_utils})

In [6]:
y_pred = model.predict_generator(gen_valid, verbose=1, workers=multiprocessing.cpu_count())
y_pred2 = 1 - y_pred[:,0]
y_pred3 = 1 - (np.argmax(y_pred, axis=-1) == 0)
print('AUC:', roc_auc_score(y_valid, y_pred2), 'ACC', accuracy_score(y_valid, y_pred3))
pprint(confusion_matrix(y_valid > 0.5, y_pred3))

102/102 [==============================] - 23s 230ms/step
